# 학습데이터, 테스트데이터 만들기

In [1]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [2]:
train_ft = pd.read_excel('train_tmp.xlsx')
test_ft = pd.read_excel('test_tmp.xlsx')

train_ft.shape,test_ft.shape

((5754, 8), (1439, 8))

In [3]:
train_ft = pd.concat([train_ft,test_ft])

In [4]:
train_color = train_ft.loc[:,['리뷰','색감_긍부정']].copy()
train_color = train_color[train_color['색감_긍부정'].isin([-1,1])]

train_color.shape

(1997, 2)

In [5]:
train_color['색감_긍부정'].value_counts() 
# 1의 비율이 훨씬 더 높은 수준임을 알 수 있음 

색감_긍부정
 1    1576
-1     421
Name: count, dtype: int64

In [6]:
train_color.columns

Index(['리뷰', '색감_긍부정'], dtype='object')

In [7]:
train_arr = train_color['리뷰'].to_numpy()

train_arr.shape

(1997,)

In [8]:
target = train_color['색감_긍부정'].to_numpy().reshape(-1,1)
target = np.where(target == -1,0,target)
target = target.astype('int64')
target.shape

(1997, 1)

In [9]:
target.mean()

0.7891837756634953

# 사전학습 모델 선정

In [10]:
model_name = 'kykim/bert-kor-base'

In [11]:
from transformers import AutoTokenizer, AutoModel

In [12]:
model = AutoModel.from_pretrained(model_name)

/home/lab01/.local/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 사전학습모델을 바탕으로 학습 실행

In [14]:
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, x, y=None):
        self.tokenizer = tokenizer
        self.x = x
        self.y = y
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        item = {}
        item["x"] = self.get_tokenizer(self.x[idx])
        if self.y is not None:
            item["y"] = torch.Tensor(self.y[idx])
        return item
    def get_tokenizer(self, text):
        x = self.tokenizer(text, padding="max_length", truncation=True)
        for k, v in x.items():
            x[k] = torch.tensor(v)
        return x

In [15]:
dt = ReviewDataset(tokenizer, train_arr, target)
dl = torch.utils.data.DataLoader(dt, batch_size=2, shuffle=False)
batch = next(iter(dl))
batch

{'x': {'input_ids': tensor([[    2, 32393, 14103,  ...,     0,     0,     0],
         [    2, 22462,  6096,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])},
 'y': tensor([[1.],
         [1.]])}

In [16]:
class Net(torch.nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.pre_model = AutoModel.from_pretrained(model_name)
        self.fc_out = torch.nn.Linear( self.pre_model.config.hidden_size, 1)

    def forward(self, x):
        x = self.pre_model(**x)
        # x[0]: 모든 시점의 히든출력 batch, seq, features
        # x[1]: CLS 토큰의 히든출력 batch, features
        return self.fc_out(x[1])

In [17]:
model = Net(model_name)
model(batch["x"])

tensor([[0.2194],
        [0.4221]], grad_fn=<AddmmBackward0>)

In [18]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    epoch_loss = 0
    model.train() # 학습 모드
    for batch in tqdm(dataloader):
        pred = model( batch["x"].to(device) )
        loss = loss_fn( pred, batch["y"].to(device) )

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(dataloader)
    return epoch_loss

In [19]:
@torch.no_grad()
def test_loop(dataloader, model, loss_fn, device):
    epoch_loss = 0
    pred_list = []
    act_func = torch.nn.Sigmoid()
    model.eval() # 평가 모드
    for batch in tqdm(dataloader):
        pred = model( batch["x"].to(device) )
        if batch.get("y") is not None:
            loss = loss_fn( pred, batch["y"].to(device) )
            epoch_loss += loss.item()

        pred = act_func(pred) # logit 값을 확률로 변환
        pred = pred.to("cpu").numpy() # cpu 이동후 ndarray 로변환
        pred_list.append(pred)

    epoch_loss /= len(dataloader)
    pred = np.concatenate(pred_list)
    return epoch_loss, pred

In [20]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

n_splits = 5
cv = KFold(n_splits, shuffle=True, random_state=42)

batch_size = 8 # 배치 사이즈
loss_fn = torch.nn.BCEWithLogitsLoss() # 손실 객체
epochs = 100 # 최대 가능한 에폭수

In [34]:
reset_seeds(42) 

# 학습용 데이터로더 객체
train_dt = ReviewDataset(tokenizer, train_arr, target)
train_dl = torch.utils.data.DataLoader(train_dt, batch_size=1, shuffle=True)

# 모델 객체와 옵티마이저 객체 생성
model = Net(model_name).to(device)
optimizer = torch.optim.Adam( model.parameters(), lr=2e-5 )
loss_fn = torch.nn.BCEWithLogitsLoss()

# epochs 조정
epochs = 6
for epoch in range(epochs):
    train_loop(train_dl, model, loss_fn, optimizer, device)
    
torch.save(model.state_dict(), "color.pth") # 모델 가중치 저장 

  0%|          | 0/1997 [00:00<?, ?it/s]

  0%|          | 0/1997 [00:00<?, ?it/s]

  0%|          | 0/1997 [00:00<?, ?it/s]

  0%|          | 0/1997 [00:00<?, ?it/s]

  0%|          | 0/1997 [00:00<?, ?it/s]

  0%|          | 0/1997 [00:00<?, ?it/s]

In [34]:
is_holdout = True
reset_seeds(42) # 재현을 위해 시드고정
best_score_list = []
for i, (tri, vai) in enumerate( cv.split(train_arr) ):
    # 학습용 데이터로더 객체
    train_dt = ReviewDataset(tokenizer, train_arr[tri], target[tri])
    train_dl = torch.utils.data.DataLoader(train_dt, batch_size=batch_size, shuffle=True)

    # 검증용 데이터로더 객체
    valid_dt = ReviewDataset(tokenizer, train_arr[vai], target[vai])
    valid_dl = torch.utils.data.DataLoader(valid_dt, batch_size=batch_size, shuffle=False)

    # 모델 객체와 옵티마이저 객체 생성
    model = Net(model_name).to(device)
    optimizer = torch.optim.Adam( model.parameters(), lr=2e-5 )

    best_score = 0 # 현재 최고 점수
    patience = 0 # 조기 종료 조건을 주기 위한 변수
    for epoch in range(epochs):
        train_loss = train_loop(train_dl, model, loss_fn, optimizer, device)
        valid_loss, pred = test_loop(valid_dl, model, loss_fn, device)

        pred = (pred > 0.5).astype(int) # 이진분류 문제에서 클래스 번호 결정
        score = f1_score(target[vai], pred,average = 'macro')

        print(train_loss, valid_loss, score)
        if score > best_score:
            best_score = score # 최고 점수 업데이트
            patience = 0
            torch.save(model.state_dict(), f"color_{i}.pth") # 최고 점수 모델 가중치 저장

        patience += 1
        if patience == 5:
            break

    print(f"{i}번째 폴드 최고 정확도: {best_score}")
    best_score_list.append(best_score)

    if is_holdout:
        break

  0%|          | 0/113 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

0.4287077569592316 0.2027646996851625 0.8865826538176427


  0%|          | 0/113 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

...

  0%|          | 0/29 [00:00<?, ?it/s]

0.02733266369781696 0.24696865790435274 0.8808262711864407


  0%|          | 0/113 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

0.026570844158027487 0.35074146282618285 0.875
0번째 폴드 최고 정확도: 0.8865826538176427


# 테스트 데이터 예측 

In [22]:
test_dt = ReviewDataset(tokenizer,train_arr)
test_dl = torch.utils.data.DataLoader(test_dt, batch_size=1, shuffle=False)

In [24]:
pred_list = []
model = Net(model_name).to(device)
state_dict = torch.load('color.pth')
model.load_state_dict(state_dict)

_, pred = test_loop(test_dl, model, loss_fn, device)

pred_list.append(pred)

/home/lab01/.local/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


  0%|          | 0/1997 [00:00<?, ?it/s]

In [25]:
pred = np.mean(pred_list, axis=0)
pred = (pred > 0.5).astype(int)

In [27]:
# 딱히 문제가 없음을 확인할 수 있는 것이다. 
pred.mean()

0.7886830245368052

In [ ]:
score = f1_score(target_test, pred,average = 'macro')
score